In [1]:
 # gpu
# 드라이브 연동
# 경로 이동
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning_AI2(2310)

/content/drive/MyDrive/Colab Notebooks/DeepLearning_AI2(2310)


### 목표
 - mlp. cnn 모델을 활용하여 이미지 분류 실습을 진행해보자
 - 이진 분류 문제를 해결해보자
 - 성능을 높이기 위한 노력을 진행해보자
     - 이미지 증식(확장)
     - 전이학습

In [2]:
# 환경세팅
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# 데이터 불러오기
data =np.load('./data/np_cat_vs_dogs.npz')
len(data) # 4개의 데이터가 있음을 확인 가능

4

In [4]:
# X_train, X_test, y_train, y_test
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

# 크기확인
print('훈련셋 :', X_train.shape, y_train.shape)
print('테스트 셋 :', X_test.shape, y_test.shape)

훈련셋 : (2000, 224, 224, 3) (2000,)
테스트 셋 : (1000, 224, 224, 3) (1000,)


In [5]:
# 환경세팅
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping # 조기 학습 중단

In [7]:
# 모델 설계 ~
mlp_model = Sequential()
# 입력층(3->1차원)
mlp_model.add(Flatten(input_shape = (224,224,3)))
# 중간층
mlp_model.add(Dense(256, activation = 'relu'))
mlp_model.add(Dense(128, activation = 'relu'))
mlp_model.add(Dense(64, activation = 'relu'))

# 출력층(이진분류)
mlp_model.add(Dense(1, activation = 'sigmoid'))

# 모델 정보 요약
mlp_model.summary()

# 컴파일
mlp_model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])

# 조기학습중단
# 교차검증 정확도를 기준으로 5번 정도 갱신이 되지 않는다면 학습중단
# 모니터=교차검증 정확도, 인내하는 정도 =5
f_ea = EarlyStopping(monitor = 'val_accuracy',
                     patience = 5,
                     verbose = 1)

# 학습
# 검증 데이터 분리 30%
# 반복횟수 40
# 배치사이즈 64
# 조기학습 중단 연결
mlp_his = mlp_model.fit(X_train, y_train,
                        validation_split = 0.3,
                        epochs = 40,
                        batch_size = 64,
                        callbacks =[f_ea])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38576641 (147.16 MB)
Trainable params: 38576641 (147.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
1

KeyboardInterrupt: ignored

In [ ]:
# mlp_his 시각화
plt.figure(figsize = (15,3))
plt.plot(mlp_his.history['accuracy'], label ='mlp acc')
plt.plot(mlp_his.history['val_accuracy'], label = 'mlp val_acc')
plt.legend()
plt.show()

In [ ]:
# 모델이 학습을 제대로 하지 못한 상황
# 복잡한 데이터, 모델 단순한 상황
# mlp 특성상 이미지에 대한 학습 성능 떨어지는게 사실
# 이미지에 대한 파악을 잘할 수 있게 해보자  > 중요한 특성을 추출 학습 > cnn

#### CNN 모델 생성
 - 1. 모델 설계
     - 뼈대 구축
     - 특성추출부(합성곱 CONV, Pooling)
     - 분류부(전결합층, 완전연결층, mlp(이진분류))
 - 2. 모델 컴파일
 - 3. 학습 및 시각화
 - 4. 예측 및 평가

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
# cnn_model
cnn_model = Sequential()
# 특성추출부 (Conv, Pooling)

cnn_model.add(Conv2D(filters = 32, kernel_size = (3,3),
                     input_shape = (224,224,3), padding ='same',
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2,2)))

cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3),
                     input_shape = (224,224,3), padding ='same',
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2,2)))
# 데이터 안의 대상의 위치가 미묘하게 바뀌어도 지역 내의 최대값은 동일

# 분류부(전결합층, mlp)
# 1차원으로 펴주는 작업 필요
cnn_model.add(Flatten())

# 중간층 1~2정도 얕게 쌓아줌
cnn_model.add(Dense(512, activation =  'relu'))

# 출력층 이진분류
cnn_model.add(Dense(1, activation  = 'sigmoid'))

# 모델 정보 확인
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     36992     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 401408)            0         
                                                                 
 dense_4 (Dense)             (None, 512)              

In [10]:
# 컴파일

# 컴파일
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_his = cnn_model.fit(X_train, y_train, validation_split=0.3, epochs=40, batch_size=64, callbacks=[f_ea])

Epoch 1/40


KeyboardInterrupt: ignored

In [ ]:
# 시각화
# cnn_his 시각화
plt.figure(figsize = (15,3))
plt.plot(cnn_his.histroy['accuracy'], label = 'cnn acc')
plt.plot(cnn_his.histroy['val_accuracy'], label = 'cnn val_acc')
plt.legend()
plt.show()

In [ ]:
# 성능이 일반화가 mlp보다는 더 된건 사실이지만
# train에 대해 너무 맞춰서 학습하는 과대적합 현상이 심화됨
# 과대적합을 해소하면서 모델의 일반화 성능을 더 높여봐야겠음

#### 데이터 이미지 증식을 통한 cnn 모델 성능 개선
 - tensorflow의 ImageDataGenerator 도구를 활용
 - 이미지 증식을 위한 설정값을 이해해보자
 - filtered/train,test 실제 이미지 경로를 연결해서 증식에 활용

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/DeepLearning_AI2(2310)'

In [13]:
# 증식할 데이터 경로 설정
train_dir = './data/cats_and_dogs_filtered/train'
test_dir = './data/cats_and_dogs_filtered/test'

# 변형될 형식 설정
# 0~ 255 펴햔 -> 0~1 분산으로 줄임
train_gen = ImageDataGenerator(rescale = 1./255,
                               zoom_range = 0.2,
                               horizontal_flip = True)

# 이미지 증식 train 에 대해서만 설정 why? 과대적합해소하기 위한 데이터는 학습용 데이터이기 때문이다.
test_gen = ImageDataGenerator(rescale = 1./255)

In [14]:
from numpy.matrixlib import test

# 데이터와 변형 설정값 연결하기
train_generator = train_gen.flow_from_directory(train_dir,
                                                target_size = (224,224),
                                                batch_size = 10,
                                                class_mode = 'binary')
# 이진분류 설정 연결 -> cats 0, dogs 1(알파벳 순서대로 읽어들이고 답을 생성)
test_generator = test_gen.flow_from_directory(test_dir,
                                              target_size = (224,224),
                                              batch_size = 10,
                                              class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
# Dropout import
from tensorflow.keras.layers import Dropout

In [16]:
# ctrl + shift + L
# cnn_model2
cnn_model2 = Sequential()
# 특성추출부 (Conv, Pooling)

cnn_model2.add(Conv2D(filters = 32, kernel_size = (3,3),
                     input_shape = (224,224,3), padding ='same',
                     activation = 'relu'))

cnn_model2.add(MaxPooling2D(pool_size = (2,2)))

cnn_model2.add(Conv2D(filters = 128, kernel_size = (3,3),
                     input_shape = (224,224,3), padding ='same',
                     activation = 'relu'))

cnn_model2.add(MaxPooling2D(pool_size = (2,2)))
# 데이터 안의 대상의 위치가 미묘하게 바뀌어도 지역 내의 최대값은 동일

# 분류부(전결합층, mlp)
# 1차원으로 펴주는 작업 필요
cnn_model2.add(Flatten())

# 중간층 1~2정도 얕게 쌓아줌
cnn_model2.add(Dense(512, activation =  'relu'))
cnn_model2.add(Dropout(0,2)) # 20% 특성은 제외(랜덤하게 비활성화)하고 학습 진행

# 출력층 이진분류
cnn_model2.add(Dense(1, activation  = 'sigmoid'))

# 모델 정보 확인
cnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     36992     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 401408)            0         
                                                                 
 dense_6 (Dense)             (None, 512)              

In [17]:
# 컴파일
cnn_model2.compile(loss = 'binary_crossentropy',
                   optimizer = 'adam',
                   metrics = ['accuracy'])

# 학습 - 이미지 증식 설정 연결
# 이미지 증식하고 학습을 진행할 경우 -> 학습현황이 들쭉날쭉한 경향이 잇음
# 조기학습 중단 x
# 반복횟수 크게 설정하는 경우 많음
# 증식 + 학습 : fit_generator()
cnn_his2 = cnn_model2.fit_generator(generator = train_generator,
                                    validation_data = test_generator,
                                    epochs = 20)

<ipython-input-17-d4e90fb1d94c>:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn_his2 = cnn_model2.fit_generator(generator = train_generator,


Epoch 1/20
 65/200 [========>.....................] - ETA: 12:39 - loss: 1.1582 - accuracy: 0.5062

KeyboardInterrupt: ignored

In [ ]:
# 시각화
plt.figure(figsize = (15,3))
plt.plot(cnn_his2.histroy['accuracy'], label = 'cnn2 acc')
plt.plot(cnn_his2.histroy['val_accuracy'], label = 'cnn2 val_acc')
plt.legend()
plt.show()

In [ ]:
# 이미지 증식을 통해 나름대로 과대적합이 해소된 것을 확인할 수 있음
# 이미지 증식으로 모델 성능 개선
# 이미지 증식 단점은 학습 시간이 오래 걸림
# 자체적으로 설계한 모델은 검증 여러번 진행하고 수정하는 작업 필요 > 시간 오래 걸림

#### 전이학습
 - 다른 사람이 만들어 놓은 모델을 사용해보자!
     - 학습 빠르게 수행할 수 있음
     - 학습할 데이터에 대해 특징을 추출하는 학습 시간이 필요가 없게 됨(시간단축)
 - 작은 데이터셋에 대해 학습할 때 과대적합 예방 가능
     - 미세조정을 하게 되면 학습하는 가중치가 줄어 과하게 학습이 진행되지 않음
 - 완전히 똑같은 데이터로 학습 x , 비슷한 분류 문제를 푼 모델을 사용하는 것(레이블이 조금은 다를 수 있음)
 - 현재 우리 목표 : 개 vs 고양이
 - 사전학습 모델 : 사자 vs 호랑이 vs 코끼리..
 - 분류할 대상은 다르지만, 판단하는 기준은 비슷할 수 있어서 가져와서 활용

In [47]:
# 데이터 확인
print('훈렷셋', X_train.shape, y_train.shape)
print('테스트셋', X_test.shape, y_test.shape)

훈렷셋 (2000, 224, 224, 3) (2000,)
테스트셋 (1000, 224, 224, 3) (1000,)


In [48]:
from tensorflow.keras.applications import VGG16

In [49]:
# VGG16: 사전에 학습 모델(Pretrained Model)
# 사전 학습된 모델을 연결해서 새롭게 학습 시키는 것 -> 전이학습:미세조정방식
# 모델의 특성 추출부만 불러오기
vgg16 = VGG16(include_top = False,
              weights = 'imagenet',
              input_shape = (224,224,3))
# top: 분류부기층(mlp, 전결합층), vgg16모델은 다중분류(1000개)이므로 사용하지 않음
# weigths: 'imagenet'> 경진대회 데이터셋을 보며 학습한 가중치 불러옴

In [50]:
# conv 미세조정 설정
# 모든 conv층 동결, 가중치 갱신 막자 > 학습 불가능하게 설정
# vgg16 100만장의 사진으로 학습 되어 있음. 성능이 좋음
# 학습시 가중치가 갱신되도록 설정해놓으면 기존에 학습된 내용이 훼손 > 성능 떨어짐
vgg16.trainable = False

In [51]:
# vgg16 + 2개 클래스 분류하는 분류기층
transfer_model = Sequential()
transfer_model.add(vgg16)

# 분류기층 > mlp(이진분류: 개 vs 고양이)
transfer_model.add(Flatten()) # 1차원 펴줌
transfer_model.add(Dense(64, activation = 'relu'))
transfer_model.add(Dense(1, activation = 'sigmoid'))

# 정보 요약
transfer_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_6 (Flatten)         (None, 25088)             0         
                                                                 
 dense_14 (Dense)            (None, 64)                1605696   
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 16320449 (62.26 MB)
Trainable params: 1605761 (6.13 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [55]:
# 학습/평가 방법 설정
transfer_model.compile(loss = 'binary_crossentropy', optimizer ='adam', metrics =['accuarcy'])

In [56]:
# 미세조정 설정
# 마지막 컨볼루션층만 학흡가능하게 설정
# 마지막 conv층만 개, 고양이 이미지 학습 가능하게 설정
# 해당 층 이름 접근
for layer in vgg16.layers:
  # block5_conv3 학습 가능하게 열어줌
  if layer.name == "block5_conv3":
    layer.trainable = True
    # 나머지 층은 학습 불가능하게 동결 설정
  else:
    layer.trainable = False

In [58]:
# 학습
trans_his = transfer_model.fit(X_train, y_train,
                               validation_split = 0.3,
                               epochs= 10)

Epoch 1/10
40/44 [==========================>...] - ETA: 0s

ValueError: ignored

In [ ]:
# 시각화
plt.figure(figsize = (15,3))
plt.plot(cnn_his.histroy['accuracy'], label = 'trans acc')
plt.plot(cnn_his.histroy['val_accuracy'], label = 'trans val_acc')
plt.legend()
plt.show()

# 이미지 증식을 통해 성능을 개선한 것보다
# 시간적으로 단축되고, val_acc 대해서 더 높은 성능을 보이고 있음
# 자체적으로 설계하는 것보다 사전 학습된 모델을 가져와서 활용해보는 것도
# 좋은 방법 중 하나